In [1050]:
case_no = 44
candidate_model = "gpt-mini"
case_no_string = str(case_no).zfill(2)
eval_model_case = candidate_model + "-" + case_no_string


In [1051]:
import pandas as pd

case_dir = "Data/cases/"
output_dir = "Data/output/"
eval_dir = "Data/evaluation/"

history = pd.read_csv(output_dir + "history/" + eval_model_case + ".csv", index_col=0)
physical = pd.read_csv(output_dir + "physical/" + eval_model_case + ".csv", index_col=0)
investigations = pd.read_csv(output_dir + "investigations/" + eval_model_case + ".csv", index_col=0)
diagnosis = pd.read_csv(output_dir + "diagnosis/" + eval_model_case + ".csv", index_col=0)

history_taking_checklist = pd.read_csv(case_dir + "history/" + str(case_no) + ".csv", index_col=0)
physical_checklist = pd.read_csv(case_dir + "physical/" + str(case_no) + ".csv", index_col=0)
investigations_checklist = pd.read_csv(case_dir + "investigations/" + str(case_no) + ".csv", index_col=0)
diagnosis_checklist = pd.read_csv(case_dir + "diagnosis/" + str(case_no) + ".csv", index_col=0)



In [1052]:
from importlib import reload
from simulator import agents
from simulator import retrievers
import math
reload(agents)
reload(retrievers)

doctor = history['Message']['HumanMessage']
patient = history['Message']['AIMessage']

conversation = []
for d, p in zip(doctor, patient):
  d = d.replace("\n", " ")
  p = str(p).replace("\n", " ")
  conversation.append(f"Doctor: {d} | Patient: {p}")
conversation_flat = "\n".join(conversation)

simulatedExaminer = agents.ExaminerAgent(model_name="gpt-4o-mini")

output = simulatedExaminer.evaluate_history_taking(conversation_flat, history_taking_checklist)


In [1053]:
history_evaluation = pd.DataFrame([(output_item.score, output_item.explanation) for output_item in output], columns=["Score", "Explanation"])
history_evaluation.to_csv(eval_dir + "history/" + eval_model_case + ".csv")

In [1054]:
from importlib import reload
from simulator import agents
from simulator import retrievers
reload(agents)
reload(retrievers)

simulatedExaminer = agents.ExaminerAgent(model_name="gpt-4o-mini")

physical_exam_suggestions = ""
for _, row in physical.iterrows():
  physical_exam_suggestions += "Suggestion: " + row['Physical Exam'] + " | Justification: " + row["Justification"] + "\n"

output = simulatedExaminer.evaluate_physical_exam(physical_exam_suggestions, physical_checklist)

In [1055]:
physical_evaluation = pd.DataFrame([(output_item.score, output_item.explanation) for output_item in output], columns=["Score", "Explanation"])
physical_evaluation.to_csv(eval_dir + "physical/" + eval_model_case + ".csv")

In [1056]:
from importlib import reload
from simulator import agents
from simulator import retrievers
reload(agents)
reload(retrievers)

simulatedExaminer = agents.ExaminerAgent(model_name="gpt-4o-mini")

investigations_suggestions = ""
for _, row in investigations.iterrows():
  investigations_suggestions += "Suggestion: " + row['Investigation'] + " | Justification: " + row["Justification"] + "\n"

output = simulatedExaminer.evaluate_investigations(investigations_suggestions, investigations_checklist)

In [1057]:
investigations_evaluation = pd.DataFrame([(output_item.score, output_item.explanation) for output_item in output], columns=["Score", "Explanation"])
investigations_evaluation.to_csv(eval_dir + "investigations/" + eval_model_case + ".csv")

In [1058]:
from importlib import reload
from simulator import agents
from simulator import retrievers
reload(agents)
reload(retrievers)

simulatedExaminer = agents.ExaminerAgent(model_name="gpt-4o-mini")

diagnosis_suggestions = ""
for _, row in diagnosis.iterrows():
  diagnosis_suggestions += row['Diagnosis'] + "\n"

output = simulatedExaminer.evaluate_ddx(diagnosis_suggestions, diagnosis_checklist)

In [1059]:
diagnosis_evaluation = pd.DataFrame([(output_item.score, output_item.explanation) for output_item in output], columns=["Score", "Explanation"])
diagnosis_evaluation.to_csv(eval_dir + "diagnosis/" + eval_model_case + ".csv")